## Object Oriented Decorator (Classic)

In [1]:
from math import sqrt


def is_prime(candidate_number: int) -> bool:
    """Check if a number is prime by testing divisibility up to its square root"""
    if candidate_number <= 1:
        return False

    # Check for divisors from 2 to square root of the number
    for potential_divisor in range(2, int(sqrt(candidate_number)) + 1):
        if candidate_number % potential_divisor == 0:
            return False  # Found a divisor, so it's not prime

    return True  # No divisors found, so it's prime

In [2]:
from abc import ABC, abstractmethod


class PrimeCounterInterface(ABC):
    """Base interface for prime counting components"""

    @abstractmethod
    def count_primes(self, upper_limit: int) -> int:
        """Count prime numbers up to the given upper limit"""
        pass


class BasicPrimeCounter(PrimeCounterInterface):
    """A simple implementation that counts prime numbers"""

    def count_primes(self, upper_limit: int) -> int:
        prime_count = 0
        for current_number in range(upper_limit):
            if is_prime(current_number):
                prime_count += 1
        return prime_count

In [3]:
from loguru import logger

prime_counter = BasicPrimeCounter()
result = prime_counter.count_primes(100000)
logger.info(f"Found {result} prime numbers")

2025-09-19 18:42:15.974 | INFO     | __main__:<module>:5 - Found 9592 prime numbers


In [4]:
from loguru import logger
from time import perf_counter


class PrimeCounterDecorator(PrimeCounterInterface):
    """Base decorator class that wraps another prime counter"""

    def __init__(self, wrapped_counter: PrimeCounterInterface):
        self._wrapped_counter = wrapped_counter


class TimingDecorator(PrimeCounterDecorator):
    """Decorator that measures and logs execution time"""

    def count_primes(self, upper_limit: int) -> int:
        start_time = perf_counter()
        prime_count = self._wrapped_counter.count_primes(upper_limit)
        end_time = perf_counter()
        execution_time = end_time - start_time
        logger.info(f"Execution time: {execution_time:.4f} seconds")
        return prime_count

In [5]:
basic_counter = BasicPrimeCounter()
timed_counter = TimingDecorator(basic_counter)
prime_count = timed_counter.count_primes(100000)
logger.info(f"Found {prime_count} prime numbers")

2025-09-19 18:42:16.102 | INFO     | __main__:count_primes:20 - Execution time: 0.1190 seconds
2025-09-19 18:42:16.103 | INFO     | __main__:<module>:4 - Found 9592 prime numbers


In [6]:
class LoggingDecorator(PrimeCounterDecorator):
    """Decorator that logs when operations start and finish"""

    def count_primes(self, upper_limit: int) -> int:
        class_name = self._wrapped_counter.__class__.__name__
        logger.info(f"Starting prime counting with {class_name}")
        prime_count = self._wrapped_counter.count_primes(upper_limit)
        logger.info(f"Finished prime counting with {class_name}")
        return prime_count

In [7]:
# Example of stacking decorators - logging + timing
basic_counter = BasicPrimeCounter()
logged_counter = LoggingDecorator(basic_counter)
timed_and_logged_counter = TimingDecorator(logged_counter)
prime_count = timed_and_logged_counter.count_primes(100000)
logger.info(f"Final result: {prime_count} prime numbers found")

2025-09-19 18:42:16.112 | INFO     | __main__:count_primes:6 - Starting prime counting with BasicPrimeCounter
2025-09-19 18:42:16.197 | INFO     | __main__:count_primes:8 - Finished prime counting with BasicPrimeCounter
2025-09-19 18:42:16.197 | INFO     | __main__:count_primes:20 - Execution time: 0.0856 seconds
2025-09-19 18:42:16.198 | INFO     | __main__:<module>:6 - Final result: 9592 prime numbers found


In [8]:
# Example of stacking decorators - timing + logging
basic_counter = BasicPrimeCounter()
timed_and_logged_counter = TimingDecorator(basic_counter)
logged_counter = LoggingDecorator(timed_and_logged_counter)
prime_count = logged_counter.count_primes(100000)
logger.info(f"Final result: {prime_count} prime numbers found")

2025-09-19 18:42:16.202 | INFO     | __main__:count_primes:6 - Starting prime counting with TimingDecorator
2025-09-19 18:42:16.284 | INFO     | __main__:count_primes:20 - Execution time: 0.0807 seconds
2025-09-19 18:42:16.284 | INFO     | __main__:count_primes:8 - Finished prime counting with TimingDecorator
2025-09-19 18:42:16.284 | INFO     | __main__:<module>:6 - Final result: 9592 prime numbers found


## Functional Decorator

In [9]:
from math import sqrt


def is_prime(candidate_number: int) -> bool:
    if candidate_number <= 1:
        return False

    for potential_divisor in range(2, int(sqrt(candidate_number)) + 1):
        if candidate_number % potential_divisor == 0:
            return False

    return True


def count_primes(upper_limit: int) -> int:
    prime_count = 0
    for current_number in range(upper_limit):
        if is_prime(current_number):
            prime_count += 1
    return prime_count

### Approach 1

In [10]:
def count_primes_with_timing(upper_limit: int) -> int:
    start_time = perf_counter()
    prime_count = count_primes(upper_limit)
    end_time = perf_counter()
    execution_time = end_time - start_time
    logger.info(f"Execution time: {execution_time:.4f} seconds")
    return prime_count

In [11]:
result = count_primes_with_timing(100000)
logger.info(f"Found {result} prime numbers")

2025-09-19 18:42:16.422 | INFO     | __main__:count_primes_with_timing:6 - Execution time: 0.1238 seconds
2025-09-19 18:42:16.423 | INFO     | __main__:<module>:2 - Found 9592 prime numbers


### Approach 2: using wrapper

In [12]:
from typing import Callable, Any


def timing(func: Callable[..., Any]) -> Callable[..., Any]:
    """Decorator that measures and logs execution time"""

    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start_time = perf_counter()
        result = func(*args, **kwargs)
        end_time = perf_counter()
        execution_time = end_time - start_time
        logger.info(f"Execution time: {execution_time:.4f} seconds")
        return result

    return wrapper

In [13]:
wrapper = timing(count_primes)
result = wrapper(100000)
logger.info(f"Found {result} prime numbers")

2025-09-19 18:42:16.514 | INFO     | __main__:wrapper:12 - Execution time: 0.0819 seconds
2025-09-19 18:42:16.515 | INFO     | __main__:<module>:3 - Found 9592 prime numbers


### Approach 3: cleaner solution

In [14]:
from typing import Callable, Any


def with_timing(func: Callable[..., Any]) -> Callable[..., Any]:
    """Decorator that measures and logs execution time"""

    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start_time = perf_counter()
        result = func(*args, **kwargs)
        end_time = perf_counter()
        execution_time = end_time - start_time
        logger.info(f"Execution time: {execution_time:.4f} seconds")
        return result

    return wrapper


@with_timing
def count_primes(upper_limit: int) -> int:
    prime_count = 0
    for current_number in range(upper_limit):
        if is_prime(current_number):
            prime_count += 1
    return prime_count


In [15]:
result = count_primes(100000)
logger.info(f"Found {result} prime numbers")

2025-09-19 18:42:16.646 | INFO     | __main__:wrapper:12 - Execution time: 0.1210 seconds
2025-09-19 18:42:16.646 | INFO     | __main__:<module>:2 - Found 9592 prime numbers


Now include loggging

In [16]:
def with_logging(func: Callable[..., Any]) -> Callable[..., Any]:
    """Decorator that measures and logs execution time"""

    def wrapper(*args: Any, **kwargs: Any) -> Any:
        logger.info(f"Starting {func.__name__}")
        result = func(*args, **kwargs)
        logger.info(f"Finished {func.__name__}")
        return result

    return wrapper


@with_logging
@with_timing
def count_primes(upper_limit: int) -> int:
    prime_count = 0
    for current_number in range(upper_limit):
        if is_prime(current_number):
            prime_count += 1
    return prime_count

In [17]:
result = count_primes(100000)
logger.info(f"Found {result} prime numbers")

2025-09-19 18:42:16.657 | INFO     | __main__:wrapper:5 - Starting wrapper
2025-09-19 18:42:16.739 | INFO     | __main__:wrapper:12 - Execution time: 0.0819 seconds
2025-09-19 18:42:16.740 | INFO     | __main__:wrapper:7 - Finished wrapper
2025-09-19 18:42:16.740 | INFO     | __main__:<module>:2 - Found 9592 prime numbers


### Approach 4: using functools wraps

In [18]:
from typing import Callable, Any
from functools import wraps


def with_timing(func: Callable[..., Any]) -> Callable[..., Any]:
    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start_time = perf_counter()
        result = func(*args, **kwargs)
        end_time = perf_counter()
        execution_time = end_time - start_time
        logger.info(f"Execution time: {execution_time:.4f} seconds")
        return result

    return wrapper


def with_logging(func: Callable[..., Any]) -> Callable[..., Any]:
    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        logger.info(f"Starting {func.__name__}")
        result = func(*args, **kwargs)
        logger.info(f"Finished {func.__name__}")
        return result

    return wrapper


@with_logging
@with_timing
def count_primes(upper_limit: int) -> int:
    prime_count = 0
    for current_number in range(upper_limit):
        if is_prime(current_number):
            prime_count += 1
    return prime_count

In [19]:
result = count_primes(100000)
logger.info(f"Found {result} prime numbers")

2025-09-19 18:42:16.751 | INFO     | __main__:wrapper:21 - Starting count_primes
2025-09-19 18:42:16.875 | INFO     | __main__:wrapper:12 - Execution time: 0.1231 seconds
2025-09-19 18:42:16.876 | INFO     | __main__:wrapper:23 - Finished count_primes
2025-09-19 18:42:16.876 | INFO     | __main__:<module>:2 - Found 9592 prime numbers
